### Libraries

In [82]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_samples, silhouette_score
from goatools.obo_parser import GODag
from scipy.stats import chi2_contingency
from scipy.stats import hypergeom
from statsmodels.stats.multitest import multipletests

import warnings
warnings.filterwarnings("ignore")

### Data loading

In [6]:
oe_gene_go_data = pd.read_csv('../data/OE_RNA_scaffolded_interproscan.csv')
go_dag = GODag("../data/go-basic.obo")
log_oe_mean_tpm_counts_by_stage = pd.read_csv('../data/log_oe_mean_tpm_counts_by_stage_clusters.csv', index_col=0)
log_oe_mean_tpm_counts_by_stage.index = log_oe_mean_tpm_counts_by_stage.index.str.replace(r'-v\d+\.\d+\.a\d+$', '', regex=True)

# Function to get the first GO term and map to biological process
def get_biological_process(go_terms):
    if pd.isna(go_terms):
        return None
    first_go = go_terms.split(',')[0].strip()
    if first_go in go_dag:
        return go_dag[first_go].name
    else:
        return None

oe_gene_go_data['biological_process'] = oe_gene_go_data['GO Terms'].apply(get_biological_process)

# Remove ".m01" from "mRNA" column
oe_gene_go_data['mRNA'] = oe_gene_go_data['mRNA'].str.replace(r'\.m\d+$', '', regex=True)

# Group by "mRNA" and keep the first "biological_process" in each group
oe_go_df = oe_gene_go_data.groupby('mRNA').first().reset_index()

# Select only the "mRNA" and "biological_process" columns
oe_go_df = oe_go_df[['mRNA', 'biological_process']]

# Ensure the index is a column for merging
log_oe_mean_tpm_counts_by_stage.reset_index(inplace=True)

# Rename the index column to match 'mRNA'
log_oe_mean_tpm_counts_by_stage.rename(columns={'index': 'mRNA'}, inplace=True)

# Merge the dataframes
merged_df = log_oe_mean_tpm_counts_by_stage.merge(oe_go_df, on='mRNA', how='left')

# Set the index back to 'mRNA'
merged_df.set_index('mRNA', inplace=True)

../data/go-basic.obo: fmt(1.2) rel(2024-06-17) 45,494 Terms


In [17]:
import pandas as pd
from goatools.obo_parser import GODag

# Assuming you have already loaded your dataframes and GO DAG
oe_gene_go_data = pd.read_csv('../data/OE_RNA_scaffolded_interproscan.csv')
go_dag = GODag("../data/go-basic.obo")
log_oe_mean_tpm_counts_by_stage = pd.read_csv('../data/log_oe_mean_tpm_counts_by_stage_clusters.csv', index_col=0)
log_oe_mean_tpm_counts_by_stage.index = log_oe_mean_tpm_counts_by_stage.index.str.replace(r'-v\d+\.\d+\.a\d+$', '', regex=True)

# Function to get the first biological process GO term
def get_biological_process(go_terms):
    if pd.isna(go_terms):
        return None
    
    go_list = go_terms.split(',')
    for go in go_list:
        go = go.strip()
        if go in go_dag:
            go_term = go_dag[go]
            if 'biological_process' in go_term.namespace:
                return go_term.name
    
    return None

# Apply the function to get biological processes for each mRNA entry
oe_gene_go_data['biological_process'] = oe_gene_go_data['GO Terms'].apply(get_biological_process)

# Remove ".m01" from "mRNA" column
oe_gene_go_data['mRNA'] = oe_gene_go_data['mRNA'].str.replace(r'\.m\d+$', '', regex=True)

# Group by "mRNA" and keep the first "biological_process" in each group
oe_go_df = oe_gene_go_data.groupby('mRNA').first().reset_index()

# Select only the "mRNA" and "biological_process" columns
oe_go_df = oe_go_df[['mRNA', 'biological_process']]

# Ensure the index is a column for merging
log_oe_mean_tpm_counts_by_stage.reset_index(inplace=True)

# Rename the index column to match 'mRNA'
log_oe_mean_tpm_counts_by_stage.rename(columns={'index': 'mRNA'}, inplace=True)

# Merge the dataframes
merged_df = log_oe_mean_tpm_counts_by_stage.merge(oe_go_df, on='mRNA', how='left')

# Set the index back to 'mRNA'
merged_df.set_index('mRNA', inplace=True)

# Now merged_df contains your data with the first biological process GO term mapped for each mRNA entry


../data/go-basic.obo: fmt(1.2) rel(2024-06-17) 45,494 Terms


In [18]:
merged_df

,3,5,E,L,A,Cluster,biological_process
mRNA,,,,,,,
62541-Oe.00g000010,0.000000,0.000000,0.949648,3.566661,0.000000,0,NaN
62541-Oe.00g000020,0.685949,0.212495,3.630567,5.938120,1.784253,5,None
62541-Oe.00g000030,0.709862,1.565167,4.552811,5.563898,3.401441,3,nucleoside transmembrane transport
62541-Oe.00g000040,0.465884,0.247251,2.718008,3.702166,1.207383,2,microtubule-based movement
62541-Oe.00g000050,0.097851,0.000000,2.325445,3.290648,2.910607,2,None
...,...,...,...,...,...,...,...
62541-Oe.00g026290,0.000000,0.000000,1.746782,2.502745,0.000000,7,NaN
62541-Oe.00g026300,0.000000,0.000000,3.226933,4.148941,2.023224,2,None
62541-Oe.00g026310,0.000000,0.000000,2.264867,3.402752,1.383015,7,None


### Get cluster annotations

In [56]:
# Initialize an empty dictionary to store the dataframes
cluster_dfs = {}

# Loop over each unique cluster value
for cluster in merged_df['Cluster'].unique():
    # Filter the dataframe for the current cluster
    cluster_data = merged_df[merged_df['Cluster'] == cluster]
    
    # Count the occurrences of each biological process
    process_counts = cluster_data['biological_process'].value_counts().reset_index()
    
    # Rename the columns to 'biological_process' and 'count'
    process_counts.columns = ['biological_process', 'count']
    
    # Add the dataframe to the dictionary
    cluster_dfs[f'cluster_{cluster}'] = process_counts

##### Identify 

In [85]:
# Get unique biological processes across all clusters
all_processes = list(merged_df['biological_process'])
cluster_0_processes = list(cluster_dfs['cluster_0']['biological_process'])
cluster_1_processes = list(cluster_dfs['cluster_1']['biological_process'])
cluster_2_processes = list(cluster_dfs['cluster_2']['biological_process'])
cluster_3_processes = list(cluster_dfs['cluster_3']['biological_process'])
cluster_4_processes = list(cluster_dfs['cluster_4']['biological_process'])
cluster_5_processes = list(cluster_dfs['cluster_5']['biological_process'])
cluster_6_processes = list(cluster_dfs['cluster_6']['biological_process'])
cluster_7_processes = list(cluster_dfs['cluster_7']['biological_process'])

In [87]:
from scipy.stats import hypergeom
import pandas as pd

def compare_all_terms(main_list, subset_list):
    # Initialize a list to store results
    results = []
    
    # Get unique terms in the subset list
    unique_terms = set(subset_list)
    
    # Loop through each unique term
    for term in unique_terms:
        # Count occurrences in main list
        total_main = main_list.count(term)
        
        # Count occurrences in subset list
        total_subset = subset_list.count(term)
        
        # Total number of items in main list
        total_items = len(main_list)
        
        # Total number of items in subset list
        total_subset_items = len(subset_list)
        
        # Calculate hypergeometric parameters
        M = total_items  # Total number of items in population (main list)
        n = total_main   # Total successes in population (occurrences in main list)
        N = total_subset_items  # Sample size (total items in subset list)
        k = total_subset  # Number of successes in sample (occurrences in subset list)
        
        # Perform hypergeometric test
        p_value = hypergeom.sf(k - 1, M, n, N)
        
        # Append result as tuple (term, p-value) to results list
        results.append((term, p_value))
    
    # Create a pandas DataFrame from results
    results_df = pd.DataFrame(results, columns=['Term', 'P-value'])
    
    # Perform FDR correction using Benjamini-Hochberg procedure
    results_df['Adjusted P-value'] = multipletests(results_df['P-value'], method='fdr_bh')[1]
    
    # Sort DataFrame by Adjusted P-value (ascending order)
    results_df = results_df.sort_values(by='Adjusted P-value')
    
    # Sort DataFrame by P-value (ascending order)
    results_df = results_df.sort_values(by='P-value')
    
    return results_df

# Example usage:

# Call the function to get the sorted results table
cluster_0_results_table = compare_all_terms(all_processes, cluster_0_processes)
cluster_1_results_table = compare_all_terms(all_processes, cluster_1_processes)
cluster_2_results_table = compare_all_terms(all_processes, cluster_2_processes)
cluster_3_results_table = compare_all_terms(all_processes, cluster_3_processes)
cluster_4_results_table = compare_all_terms(all_processes, cluster_4_processes)
cluster_5_results_table = compare_all_terms(all_processes, cluster_5_processes)
#cluster_6_results_table = compare_all_terms(all_processes, cluster_6_processes)
cluster_7_results_table = compare_all_terms(all_processes, cluster_7_processes)

In [98]:
cluster_3_results_table

,Term,P-value,Adjusted P-value
15,protein targeting,0.012153,0.048614
28,positive regulation of translational elongation,0.012153,0.048614
17,maturation of LSU-rRNA from tricistronic rRNA ...,0.012153,0.048614
30,nucleosome assembly,0.012153,0.048614
6,lysyl-tRNA aminoacylation,0.012153,0.048614
31,"pentose-phosphate shunt, non-oxidative branch",0.012153,0.048614
4,nucleoside transmembrane transport,0.012153,0.048614
3,cell redox homeostasis,0.012153,0.048614
1,purine nucleotide biosynthetic process,0.024164,0.051549
21,regulation of DNA replication,0.024164,0.051549
